In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 
import seaborn as sns
from matplotlib import pyplot as plt

from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization, ReLU, AveragePooling2D
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, UpSampling2D
from tensorflow.keras.layers import Lambda, Subtract, Add
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils

%matplotlib inline

In [ ]:
# clf = load_model('../input/glass-classifier/glass_cf_15_sigmoid.tf')
clf = load_model('../input/smileclf/smiling_cf_15_sigmoid.tf')

In [ ]:
latent_dim = 400

In [ ]:
# create a sampling layer
from tensorflow.keras.layers import Layer
class reparameterize(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        mean, logvar = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        eps = tf.keras.backend.random_normal(shape=(batch, dim))
        return eps * tf.exp(logvar * .5) + mean

In [ ]:
# encoder
input_shape = (64, 64, 3)
kernel_size = 5
image_size = 64 # square, original to be resized
crop_size = 148 # center crop to this size per dimension
batch_size = 128
gamma_init = tf.random_normal_initializer(1., 0.02)

# First build the Encoder Model part
inputs = Input(shape=input_shape, name='encoder_input')
x = Conv2D(filters=64, kernel_size=kernel_size, strides=2, padding='same', name='conv1')(inputs)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=128, kernel_size=kernel_size, strides=2, padding='same', name='conv2')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=256, kernel_size=kernel_size, strides=2, padding='same', name='conv3')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=512, kernel_size=kernel_size, strides=2, padding='same', name='conv4')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)

shape = K.int_shape(x)

x = Flatten()(x)
mean = Dense(latent_dim, name="z_mean")(x)
logvar = Dense(latent_dim, name="z_log_var")(x)
z = reparameterize()([mean, logvar])

# Instantiate Encoder Model
encoder = Model(inputs, [z, mean, logvar], name='encoder')
# encoder.summary()

In [ ]:
# decoder
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3] * 2)(latent_inputs)
x = Reshape((shape[1]*2, shape[2]*2, shape[3]//2), name='h0_reshape')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)

x = Conv2DTranspose(filters=256, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=128, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=64, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=3, kernel_size=5, strides=1, padding='same')(x)
outputs = Activation('tanh', name='decoder_output')(x)

# Instantiate Decoder Model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

In [ ]:
class VAE(Model):
    def __init__(self, encoder, latent_dim, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.cf_layer = Dense(units=latent_dim, kernel_initializer=tf.constant_initializer(np.eye(latent_dim)))
        # self.cf_layer = Sequential(
        #     [
        #     Input(shape=(latent_dim,)),
        #     Dense(32, activation="relu"),
        #     Dense(latent_dim),
        #     ]
        # )
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z, mean, logvar = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 64 * 64
            kl_loss = 1 + logvar - tf.square(mean) - tf.exp(logvar)
            kl_loss = tf.reduce_mean(kl_loss)
#             kl_loss = tf.reduce_sum(kl_loss)
            kl_loss *= -0.5
#             LOSS_FACTOR = 10000
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, inputs):
      z, mean, logvar = encoder(inputs)
      return decoder(z)

    def cf(self, inputs):
      z, mean, logvar = encoder(inputs)
      return decoder(self.cf_layer(z))

    @tf.function
    def sample(self, eps=None):
      if eps is None:
        eps = tf.random.normal(shape=(100, latent_dim))
      return tf.sigmoid(self.decoder(eps))

# Instantiate vae
vae = VAE(encoder, latent_dim, decoder)

In [ ]:
alpha = 0.07
# vae.load_weights('../input/hyper-alpha/alpha/cgen-512-'+str(alpha)+'-100/cgen-512-'+str(alpha)+'-100/model.tf')
# vae.load_weights('../input/celeba-alpha/cgen-512-'+str(alpha)+'-200/cgen-512-'+str(alpha)+'-200/model.tf')
# vae.load_weights('../input/celeba-latentdim/dimension/cgen-'+str(latent_dim)+'-0.01-200/cgen-'+str(latent_dim)+'-0.01-200/model.tf')
vae.load_weights('../input/smile-cgen-400/cgen-smile-400-0.01-200/model.tf')

In [ ]:
# set variables 
main_folder = '../input/celeba-dataset/'
images_folder = main_folder + 'img_align_celeba/img_align_celeba/'
EXAMPLE_PIC = images_folder + '000506.jpg'

In [ ]:
# import the data set that include the attribute for each picture
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

In [ ]:
glass = df_attr["Smiling"]
# In gender array 0-no glass while 1-glass
glass.head(5)

In [ ]:
df_partition = pd.read_csv(main_folder + 'list_eval_partition.csv')
# df_partition.head()

In [ ]:
df_partition['partition'].value_counts().sort_index()

In [ ]:
df_partition.set_index('image_id', inplace=True)
df_par_attr = df_partition.join(glass, how='inner')
df_par_attr.head()

In [ ]:
df_ = df_par_attr[(df_par_attr['partition'] == 2) 
                           & (df_par_attr['Smiling'] == 0)][0:1]
# .sample(1)
# iloc[0]
df_

In [ ]:
def load_reshape_img(fname):
    x = cv2.imread(fname)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (64,64)).astype('float32') / 255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)][0:int(num_samples/2)]
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)][int(num_samples/2):num_samples]])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 64, 64, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [ ]:
TRAINING_SAMPLES = 500
IMG_WIDTH = 64
IMG_HEIGHT = 64

x_train, y_train = generate_df(2, 'Smiling', TRAINING_SAMPLES)
# x_valid, y_valid = generate_df(1, 'Eyeglasses', VALIDATION_SAMPLES)

In [ ]:
temp = 0
for i in range(len(x_train)):
    if i==0:
        temp = x_train[i]
    else:
        temp  = np.concatenate((temp, x_train[i]), axis=0)
x_train = temp

In [ ]:

# plt.imshow(x_train[1501])
plt.imshow(x_train[8])

In [ ]:
result = vae.cf(x_train[7:9])
print(result[1].shape)
plt.imshow(result[1])

In [ ]:
def generate_array(images):
    if type(images) == 'tensorflow.python.framework.ops.EagerTensor':
        images = K.eval(images)
    for i in range(len(images)):
        raw_0 = images[i]
        raw_0 = np.uint8(raw_0 * 255)
#         raw_0 = raw_0.reshape((28, 28))
#         raw_0 = np.array(Image.fromarray(raw_0).convert("RGB"))
        raw_0 = raw_0[np.newaxis, :]
        if i == 0:
            raw = raw_0
        else:
            raw = np.concatenate((raw, raw_0), axis=0)
    return raw

In [ ]:
test_ori_img = x_train[:250]
test_tar_img = x_train[250:]

recon_ori = vae(test_ori_img)
recon_ori = K.eval(recon_ori)

recon_target = vae(test_tar_img)
recon_target = K.eval(recon_target)

cf_ori = vae.cf(test_ori_img)
cf_ori = K.eval(cf_ori)

cf_tar = vae.cf(test_tar_img)
cf_tar = K.eval(cf_tar)


# clf = load_model(models_dir + 'classifier_trained_on_' + str(target_class) + '.tf')

#.............................
clf_result = clf(cf_ori)
count = 0
for i in clf_result:
  if i >= 0.5:
    count += 1
print("clfed target percentage for cf: ", count/len(clf_result))
print(count, len(clf_result))

count = 0
clf_result = clf(recon_ori)
for i in clf_result:
  if i >= 0.5:
    count += 1
print("clfed target percentage for vae: ", count/len(clf_result))
print(count, len(clf_result))

#.............example................
raw_0 = K.eval(cf_ori)[8]
raw_0 = np.uint8(raw_0 * 255)

plt.imshow(raw_0)
plt.show()

raw_ori = generate_array(test_ori_img)
raw_tar = generate_array(test_tar_img)
recon_ori = generate_array(recon_ori)
recon_tar = generate_array(recon_target)
cf_ori = generate_array(cf_ori)
cf_tar = generate_array(cf_tar)

print(cf_ori.shape)

raw_ori = raw_ori.reshape([-1,64,64,3]).transpose([0,3,1,2])
raw_tar = raw_tar.reshape([-1,64,64,3]).transpose([0,3,1,2])
recon_ori = recon_ori.reshape([-1,64,64,3]).transpose([0,3,1,2])
recon_tar = recon_tar.reshape([-1,64,64,3]).transpose([0,3,1,2])
cf_ori = cf_ori.reshape([-1,64,64,3]).transpose([0,3,1,2])
cf_tar = cf_tar.reshape([-1,64,64,3]).transpose([0,3,1,2])

recon_ori.shape

In [ ]:
!pip install tensorflow-gan

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import functools
import numpy as np
import time
from tensorflow.python.ops import array_ops
import tensorflow_gan as tfgan

session=tf.compat.v1.InteractiveSession()
# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 50

# Run images through Inception.
inception_images = tf.compat.v1.placeholder(tf.float32, [None, 3, None, None], name = 'inception_images')
activations1 = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'activations1')
activations2 = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'activations2')
fcd = tfgan.eval.frechet_classifier_distance_from_activations(activations1, activations2)

INCEPTION_TFHUB = 'https://tfhub.dev/tensorflow/tfgan/eval/inception/1'
INCEPTION_FINAL_POOL = 'pool_3'

def inception_activations(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.compat.v1.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    activations = tf.map_fn(
        fn = tfgan.eval.classifier_fn_from_tfhub(INCEPTION_TFHUB, INCEPTION_FINAL_POOL, True),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 1,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    activations = array_ops.concat(array_ops.unstack(activations), 0)
    return activations

activations =inception_activations()

def get_inception_activations(inps):
    n_batches = int(np.ceil(float(inps.shape[0]) / BATCH_SIZE))
    act = np.zeros([inps.shape[0], 2048], dtype = np.float32)
    for i in range(n_batches):
        inp = inps[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] / 255. * 2 - 1
        act[i * BATCH_SIZE : i * BATCH_SIZE + min(BATCH_SIZE, inp.shape[0])] = session.run(activations, feed_dict = {inception_images: inp})
    return act

def activations2distance(act1, act2):
    return session.run(fcd, feed_dict = {activations1: act1, activations2: act2})
        
def get_fid(images1, images2):
    session=tf.get_default_session()
    assert(type(images1) == np.ndarray)
    assert(len(images1.shape) == 4)
    assert(images1.shape[1] == 3)
    assert(np.min(images1[0]) >= 0 and np.max(images1[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(type(images2) == np.ndarray)
    assert(len(images2.shape) == 4)
    assert(images2.shape[1] == 3)
    assert(np.min(images2[0]) >= 0 and np.max(images2[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(images1.shape == images2.shape), 'The two numpy arrays must have the same shape'
    print('Calculating FID with %i images from each distribution' % (images1.shape[0]))
    start_time = time.time()
    
    act1 = get_inception_activations(images1)
    act2 = get_inception_activations(images2)
    fid = activations2distance(act1, act2)
    print('FID calculation time: %f s' % (time.time() - start_time))
    return fid

In [ ]:
DATA_DIM=28*28*3
fid_recon_ori = get_fid(raw_ori, recon_ori)
fid_cf_ori = get_fid(raw_ori, cf_ori)
# fid_cf_tar = get_fid(raw_tar, cf_ori)
print('FID ori & recon_ori: %f'% fid_recon_ori)
print('FID ori & cf_ori: %f'% fid_cf_ori)
# print('FID tar & cf_ori: %f'% fid_cf_tar)

In [ ]:
session=tf.compat.v1.InteractiveSession()
# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 50
INCEPTION_TFHUB = 'https://tfhub.dev/tensorflow/tfgan/eval/inception/1'
INCEPTION_OUTPUT = 'logits'

# Run images through Inception.
inception_images = tf.compat.v1.placeholder(tf.float32, [None, 3, None, None], name = 'inception_images')
def inception_logits(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.compat.v1.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    logits = tf.map_fn(
        fn = tfgan.eval.classifier_fn_from_tfhub(INCEPTION_TFHUB, INCEPTION_OUTPUT, True),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 8,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    logits = array_ops.concat(array_ops.unstack(logits), 0)
    return logits

logits=inception_logits()

def get_inception_probs(inps):
    session=tf.get_default_session()
    n_batches = int(np.ceil(float(inps.shape[0]) / BATCH_SIZE))
    preds = np.zeros([inps.shape[0], 1000], dtype = np.float32)
    for i in range(n_batches):
        inp = inps[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] / 255. * 2 - 1
        preds[i * BATCH_SIZE : i * BATCH_SIZE + min(BATCH_SIZE, inp.shape[0])] = session.run(logits,{inception_images: inp})[:, :1000]
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    return preds

def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

def get_inception_score(images, splits=10):
    assert(type(images) == np.ndarray)
    assert(len(images.shape) == 4)
    assert(images.shape[1] == 3)
    assert(np.min(images[0]) >= 0 and np.max(images[0]) > 10), 'Image values should be in the range [0, 255]'
    print('Calculating Inception Score with %i images in %i splits' % (images.shape[0], splits))
    start_time=time.time()
    preds = get_inception_probs(images)
    mean, std = preds2score(preds, splits)
    print('Inception Score calculation time: %f s' % (time.time() - start_time))
    return mean, std  # Reference values: 11.38 for 50000 CIFAR-10 training set images, or mean=11.31, std=0.10 if in 10 splits.

In [ ]:
# IS between training set and test set
is_cf_ori = get_inception_score(cf_ori)
is_vae_ori = get_inception_score(recon_ori)
print('is_cf_ori: ', is_cf_ori)
print('is_vae_ori: ', is_vae_ori)